In [ ]:
%cd /content/drive/MyDrive/MLPortfolio

/content/drive/MyDrive/MLPortfolio


In [ ]:
!mkdir Medical_Imaging

mkdir: cannot create directory ‘Medical_Imaging’: File exists


In [ ]:
%cd Medical_Imaging

/content/drive/MyDrive/MLPortfolio/Medical_Imaging


In [ ]:
 # Download dataset from Kaggle (upload your Kaggle API key first)
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
{"username":"hassanwaris","key":"d76b465b1559d086339107c3d0ae2059"}
User cancelled operation
Archive:  chest-xray-pneumonia.zip


In [ ]:
%cd chest_xray

/content/drive/MyDrive/MLPortfolio/Medical_Imaging/chest_xray


In [ ]:
# Install dependencies (if needed)
!pip install tensorflow matplotlib

# Import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, AUC
import matplotlib.pyplot as plt


In [ ]:
# Define dataset directories
train_data_dir = '/content/drive/MyDrive/MLPortfolio/Medical_Imaging/chest_xray/train'

# (Optional) Mount Google Drive if not already done
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Create ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.


In [ ]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze convolutional base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall(), AUC()]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_model.h5'),
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)


In [ ]:
class_weights = {0: 1.5, 1: 0.5}  # Adjust for class imbalance

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights,
    callbacks=[checkpoint_callback, early_stop, lr_scheduler]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.7623 - auc: 0.7931 - loss: 0.3941 - precision: 0.8810 - recall: 0.7954
Epoch 1: val_loss improved from inf to 0.28592, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 1305s 10s/step - accuracy: 0.7628 - auc: 0.7940 - loss: 0.3933 - precision: 0.8814 - recall: 0.7957 - val_accuracy: 0.8830 - val_auc: 0.9761 - val_loss: 0.2859 - val_precision: 0.9954 - val_recall: 0.8465 - learning_rate: 1.0000e-04
Epoch 2/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 718ms/step - accuracy: 0.8924 - auc: 0.9660 - loss: 0.1777 - precision: 0.9755 - recall: 0.8786
Epoch 2: val_loss improved from 0.28592 to 0.28028, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 119s 912ms/step - accuracy: 0.8924 - auc: 0.9661 - loss: 0.1776 - precision: 0.9755 - recall: 0.8787 - val_accuracy: 0.8821 - val_auc: 0.9776 - val_loss: 0.2803 - val_precision: 0.9895 - val_recall: 0.8503 - learning_rate: 1.0000e-04
Epoch 3/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 695ms/step - accuracy: 0.9215 - auc: 0.9790 - loss: 0.1381 - precision: 0.9827 - recall: 0.9104
Epoch 3: val_loss improved from 0.28028 to 0.20502, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 874ms/step - accuracy: 0.9214 - auc: 0.9790 - loss: 0.1382 - precision: 0.9827 - recall: 0.9104 - val_accuracy: 0.9156 - val_auc: 0.9857 - val_loss: 0.2050 - val_precision: 0.9914 - val_recall: 0.8942 - learning_rate: 1.0000e-04
Epoch 4/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - accuracy: 0.9266 - auc: 0.9825 - loss: 0.1278 - precision: 0.9844 - recall: 0.9155
Epoch 4: val_loss improved from 0.20502 to 0.15256, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 115s 879ms/step - accuracy: 0.9266 - auc: 0.9825 - loss: 0.1278 - precision: 0.9844 - recall: 0.9155 - val_accuracy: 0.9358 - val_auc: 0.9910 - val_loss: 0.1526 - val_precision: 0.9917 - val_recall: 0.9213 - learning_rate: 1.0000e-04
Epoch 5/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 694ms/step - accuracy: 0.9252 - auc: 0.9822 - loss: 0.1282 - precision: 0.9778 - recall: 0.9217
Epoch 5: val_loss did not improve from 0.15256
131/131 ━━━━━━━━━━━━━━━━━━━━ 113s 867ms/step - accuracy: 0.9253 - auc: 0.9822 - loss: 0.1281 - precision: 0.9778 - recall: 0.9217 - val_accuracy: 0.9166 - val_auc: 0.9892 - val_loss: 0.2043 - val_precision: 0.9943 - val_recall: 0.8929 - learning_rate: 1.0000e-04
Epoch 6/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - accuracy: 0.9389 - auc: 0.9862 - loss: 0.1105 - precision: 0.9823 - recall: 0.9349
Epoch 6: val_loss improved from 0.15256 to 0.15082, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 873ms/step - accuracy: 0.9389 - auc: 0.9862 - loss: 0.1105 - precision: 0.9823 - recall: 0.9349 - val_accuracy: 0.9377 - val_auc: 0.9874 - val_loss: 0.1508 - val_precision: 0.9837 - val_recall: 0.9316 - learning_rate: 1.0000e-04
Epoch 7/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 691ms/step - accuracy: 0.9400 - auc: 0.9875 - loss: 0.1073 - precision: 0.9831 - recall: 0.9358
Epoch 7: val_loss improved from 0.15082 to 0.14548, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 870ms/step - accuracy: 0.9400 - auc: 0.9875 - loss: 0.1073 - precision: 0.9831 - recall: 0.9358 - val_accuracy: 0.9453 - val_auc: 0.9901 - val_loss: 0.1455 - val_precision: 0.9931 - val_recall: 0.9329 - learning_rate: 1.0000e-04
Epoch 8/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 686ms/step - accuracy: 0.9371 - auc: 0.9869 - loss: 0.1079 - precision: 0.9832 - recall: 0.9310
Epoch 8: val_loss did not improve from 0.14548
131/131 ━━━━━━━━━━━━━━━━━━━━ 112s 856ms/step - accuracy: 0.9371 - auc: 0.9869 - loss: 0.1079 - precision: 0.9832 - recall: 0.9310 - val_accuracy: 0.9406 - val_auc: 0.9897 - val_loss: 0.1566 - val_precision: 0.9931 - val_recall: 0.9265 - learning_rate: 1.0000e-04
Epoch 9/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - accuracy: 0.9368 - auc: 0.9873 - loss: 0.1093 - precision: 0.9793 - recall: 0.9333
Epoch 9: val_loss improved from 0.14548 to 0.12657, saving model to /content/drive/MyDrive/MLPortfolio/Medical_Imaging/checkpoints/best_model.h5


131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 873ms/step - accuracy: 0.9368 - auc: 0.9873 - loss: 0.1093 - precision: 0.9794 - recall: 0.9334 - val_accuracy: 0.9473 - val_auc: 0.9916 - val_loss: 0.1266 - val_precision: 0.9865 - val_recall: 0.9419 - learning_rate: 1.0000e-04
Epoch 10/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.9419 - auc: 0.9879 - loss: 0.1065 - precision: 0.9797 - recall: 0.9423
Epoch 10: val_loss did not improve from 0.12657
131/131 ━━━━━━━━━━━━━━━━━━━━ 115s 878ms/step - accuracy: 0.9419 - auc: 0.9879 - loss: 0.1065 - precision: 0.9797 - recall: 0.9422 - val_accuracy: 0.9300 - val_auc: 0.9896 - val_loss: 0.1698 - val_precision: 0.9944 - val_recall: 0.9110 - learning_rate: 1.0000e-04
Epoch 11/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 708ms/step - accuracy: 0.9482 - auc: 0.9895 - loss: 0.0963 - precision: 0.9863 - recall: 0.9427
Epoch 11: val_loss did not improve from 0.12657
131/131 ━━━━━━━━━━━━━━━━━━━━ 115s 882ms/step - accuracy: 0.9482 - auc: 0.9895 - loss: 0.0963 - pre

131/131 ━━━━━━━━━━━━━━━━━━━━ 114s 871ms/step - accuracy: 0.9505 - auc: 0.9906 - loss: 0.0912 - precision: 0.9865 - recall: 0.9470 - val_accuracy: 0.9453 - val_auc: 0.9917 - val_loss: 0.1197 - val_precision: 0.9825 - val_recall: 0.9432 - learning_rate: 1.0000e-04
Epoch 13/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 698ms/step - accuracy: 0.9526 - auc: 0.9910 - loss: 0.0864 - precision: 0.9899 - recall: 0.9455
Epoch 13: val_loss did not improve from 0.11967
131/131 ━━━━━━━━━━━━━━━━━━━━ 113s 864ms/step - accuracy: 0.9526 - auc: 0.9910 - loss: 0.0864 - precision: 0.9899 - recall: 0.9456 - val_accuracy: 0.9463 - val_auc: 0.9914 - val_loss: 0.1421 - val_precision: 0.9918 - val_recall: 0.9355 - learning_rate: 1.0000e-04
Epoch 14/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 693ms/step - accuracy: 0.9564 - auc: 0.9912 - loss: 0.0831 - precision: 0.9935 - recall: 0.9474
Epoch 14: val_loss did not improve from 0.11967
131/131 ━━━━━━━━━━━━━━━━━━━━ 113s 867ms/step - accuracy: 0.9564 - auc: 0.9911 - loss: 0.0832 - pre

In [ ]:
# Unfreeze the top 50 layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall(), AUC()]
)

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[checkpoint_callback, early_stop, lr_scheduler]
)


In [ ]:
val_loss, val_acc, val_prec, val_rec, val_auc = model.evaluate(val_generator)
print("\nFinal Evaluation Results:")
print(f"Accuracy: {val_acc:.3f}")
print(f"Precision: {val_prec:.3f}")
print(f"Recall (Sensitivity): {val_rec:.3f}")
print(f"AUC: {val_auc:.3f}")


In [ ]:
import glob
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

def generate_gradcam(model, img_path, last_conv_layer_name=None):
    """Generate and display a Grad-CAM heatmap for a given image."""
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array / 255.0, axis=0)

    # Automatically find the last convolutional layer if not specified
    if last_conv_layer_name is None:
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                last_conv_layer_name = layer.name
                break

    last_conv_layer = model.get_layer(last_conv_layer_name)
    grad_model = tf.keras.models.Model(
        [model.inputs], [last_conv_layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    # Normalize heatmap between 0 and 1
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1

    # Display original and Grad-CAM overlay
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(img))
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(img))
    plt.imshow(heatmap, cmap="jet", alpha=0.4)
    plt.title(f"Grad-CAM (Pred: {predictions[0][0]:.2f})")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Automatically pick a random image from the validation dataset
val_dir = '/content/drive/MyDrive/MLPortfolio/Medical_Imaging/chest_xray/val'
sample_images = glob.glob(os.path.join(val_dir, '**', '*.jpeg'), recursive=True)

if len(sample_images) == 0:
    print("No validation images found. Check your path:", val_dir)
else:
    random_img = random.choice(sample_images)
    print("Running Grad-CAM for image:", random_img)
    generate_gradcam(model, random_img)


In [ ]:
import matplotlib.pyplot as plt
import os

def plot_training_curves(history, fine_tune_history=None, save_plots=False):
    """
    Plots training and validation metrics for both training and fine-tuning phases.
    Optionally saves the plots to Google Drive.
    """
    metrics = [m for m in history.history.keys() if not m.startswith('val_')]
    val_metrics = [f'val_{m}' for m in metrics if f'val_{m}' in history.history]

    # Merge fine-tuning histories if provided
    if fine_tune_history:
        for k, v in fine_tune_history.history.items():
            history.history[k] = history.history.get(k, []) + v

    # Directory to save plots (if enabled)
    if save_plots:
        save_dir = '/content/drive/MyDrive/MLPortfolio/Medical_Imaging/results/plots'
        os.makedirs(save_dir, exist_ok=True)
        print(f"📁 Saving plots to: {save_dir}")

    # Plot each metric
    for metric in metrics:
        plt.figure(figsize=(8, 5))
        epochs = range(1, len(history.history[metric]) + 1)
        plt.plot(epochs, history.history[metric], label=f'Train {metric.capitalize()}')
        if f'val_{metric}' in history.history:
            plt.plot(epochs, history.history[f'val_{metric}'], label=f'Val {metric.capitalize()}')

        plt.title(f'{metric.capitalize()} Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()

        # Save to Drive if enabled
        if save_plots:
            save_path = os.path.join(save_dir, f'{metric}_plot.png')
            plt.savefig(save_path)
            print(f"✅ Saved: {save_path}")

        plt.show()

# Example usage:
plot_training_curves(history, fine_tune_history, save_plots=True)
